### Load tensorflow

In [ ]:
import tensorflow as tf

### Read the data
<font size="2">Data for this exercise can be downloaded from http://www.manythings.org/anki/</font>

In [ ]:
#You can use wget to download the file directly
!wget https://www.manythings.org/anki/hin-eng.zip --quiet

In [ ]:
!ls -l

In [ ]:
#Unzip
!unzip hin-eng.zip

In [ ]:
!ls -l

In [ ]:
!cat hin.txt

In [ ]:
import zipfile
import io

#Read the zip file
zf = zipfile.ZipFile('hin-eng.zip', 'r')

#Extract data from zip file
data = ''
with zf.open('hin.txt') as readfile:
  for line in io.TextIOWrapper(readfile, 'utf-8'):
    data += line

In [ ]:
len(data)

In [ ]:
type(data)

In [ ]:
print(data[40000:50000])


### Extract Source and Target Language pairs

In [ ]:
#Split by newline character
data =  data.split('\n')

In [ ]:
len(data)

In [ ]:
#Show some Data
data[100:105]

### Separate Source and Target pairs

In [ ]:
encoder_text = [] #Initialize Source language list
decoder_text = [] #Initialize Target language list

#Iterate over data
for line in data:
    try:
        in_txt, out_txt, attributiontxttxt = line.split('\t')
        encoder_text.append(in_txt)

        # Add tab '<start>' as 'start sequence in target
        # And '<end>' as End
        decoder_text.append('<start> ' + out_txt + ' <end>')
    except:
        pass #ignore data which goes into error

### Separate Source and Target pairs..

In [ ]:
#English (Source)
encoder_text[100:105]

In [ ]:
#Target language
decoder_text[100:105]

In [ ]:
len(encoder_text)

In [ ]:
len(decoder_text)

### Tokenize Source language sentences

In [ ]:
#Tokenizer for source language
encoder_t = tf.keras.preprocessing.text.Tokenizer(lower=True)
encoder_t.fit_on_texts(encoder_text) #Fit it on Source sentences

In [ ]:
#Vocab
len(encoder_t.word_index)

In [ ]:
print(encoder_t.word_index)

In [ ]:
#Convert English text to indexes
encoder_seq = encoder_t.texts_to_sequences(encoder_text) #Convert sentences to numbers

In [ ]:
encoder_text[100:105]

In [ ]:
encoder_seq[100:105] #Display some converted sentences

In [ ]:
#Maximum length of sentence
max_encoder_seq_length = max([len(txt) for txt in encoder_seq])
print('Maximum sentence length for Source language: ', max_encoder_seq_length)

In [ ]:
#Source language Vocablury
encoder_vocab_size = len(encoder_t.word_index)
print('Source language vocablury size: ', encoder_vocab_size)

### Tokenize Target language sentences

In [ ]:
#Tokenizer for target language, filters should not <start> and <end>
#remove < and > used in Target language sequences
#decoder_t = tf.keras.preprocessing.text.Tokenizer()
decoder_t = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')

decoder_t.fit_on_texts(decoder_text) #Fit it on target sentences

decoder_seq = decoder_t.texts_to_sequences(decoder_text) #Convert sentences to numbers

In [ ]:
print(decoder_t.word_index)

In [ ]:
len(decoder_t.word_index)

In [ ]:
decoder_text[100:105]

In [ ]:
decoder_seq[100:105]

In [ ]:
#Maximum length of sentence
max_decoder_seq_length = max([len(txt) for txt in decoder_seq])
print('Maximum sentence length for Target language: ', max_decoder_seq_length)

#Target language Vocablury
decoder_vocab_size = len(decoder_t.word_index)
print('Target language vocablury size: ', decoder_vocab_size)

### Compare different sentences length

In [ ]:
#Source Language sentences
print('Length for sentence number 100: ', len(encoder_seq[1000]))
print('Length for sentence number 150: ', len(encoder_seq[1500]))

In [ ]:
#Target Language sentences
print('Length for sentence number 100: ', len(decoder_seq[100]))
print('Length for sentence number 150: ', len(decoder_seq[1500]))

### How do we make it same?

### Padding the sentences

In [ ]:
#Source sentences
encoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(encoder_seq,
                                                                   maxlen=max_encoder_seq_length,
                                                                   padding='pre')

#Target Sentences
decoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(decoder_seq,
                                                                   maxlen=max_decoder_seq_length,
                                                                   padding='post')

In [ ]:
print('Source data shape: ', encoder_input_data.shape)
print('Target data shape: ', decoder_input_data.shape)

In [ ]:
encoder_text[100]

In [ ]:
encoder_input_data[100]

In [ ]:
decoder_text[100]

In [ ]:
decoder_input_data[100]

#### Integer to Word converter for Decoder data

In [ ]:
int_to_word_decoder = dict((i,c) for c, i in decoder_t.word_index.items())

In [ ]:
print(int_to_word_decoder)

### Building Decoder Output

In [ ]:
decoder_input_data.shape

In [ ]:
import numpy as np

#Initialize array
decoder_target_data = np.zeros((decoder_input_data.shape[0],
                                decoder_input_data.shape[1]))

#Shift Target output by one word
for i in range(decoder_input_data.shape[0]):
    for j in range(1,decoder_input_data.shape[1]):
        decoder_target_data[i][j-1] = decoder_input_data[i][j]

In [ ]:
#decoder_t.word_index

In [ ]:
#<start> yeh kitab hai <end>
#Yeh kitab hai <end> 0

In [ ]:
decoder_text[0]

In [ ]:
decoder_input_data[0]

In [ ]:
decoder_target_data[0]

In [ ]:
decoder_target_data.shape

#### Convert target data in one hot vector

In [ ]:
#Initialize one hot encoding array
decoder_target_one_hot = np.zeros((decoder_input_data.shape[0], #number of sentences
                                   decoder_input_data.shape[1], #Number of words in each sentence
                                   len(decoder_t.word_index)+1)) #Vocab size + 1

In [ ]:
decoder_target_one_hot.shape

In [ ]:
#Build one hot encoded array
for i in range(decoder_target_data.shape[0]):
    for j in range(decoder_target_data.shape[1]):
        decoder_target_one_hot[i][j] = tf.keras.utils.to_categorical(decoder_target_data[i][j],
                                                                     num_classes=len(decoder_t.word_index)+1)

In [ ]:
decoder_target_one_hot.shape

In [ ]:
#1st example
print(decoder_target_one_hot[0][2])

### Building the Training Model

In [ ]:
#Define config parameters
encoder_embedding_size = 50
decoder_embedding_size = 50
memory_size = 128 #Memory size for LSTM

#### Build Encoder

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
#Input Layer
encoder_inputs = tf.keras.layers.Input(shape=(22,))

#Embedding layer - Word2Vec
encoder_embedding = tf.keras.layers.Embedding(encoder_vocab_size+1, #Size for One hot encoding
                                              encoder_embedding_size) #How many numbers to use for each word
#Get embedding layer output by feeding inputs
encoder_embedding_output = encoder_embedding(encoder_inputs)

#LSTM Layer and its output
x, state_h, state_c = tf.keras.layers.LSTM(memory_size,
                                           return_state=True,
                                           dropout=0.2,
                                           recurrent_dropout=0.3)(encoder_embedding_output)

#Build a list to feed Decoder - Sentence Embedding
encoder_states = [state_h, state_c]

In [ ]:
encoder_embedding_output

In [ ]:
state_h

In [ ]:
state_c

In [ ]:
encoder_states

#### Build Decoder

In [ ]:
#Decode input - padded Target sentences
decoder_inputs = tf.keras.layers.Input(shape=(27,))

#Decoder Embedding layer
decoder_embedding = tf.keras.layers.Embedding(decoder_vocab_size + 1,
                                              decoder_embedding_size)

#Embedding layer output
decoder_embedding_output = decoder_embedding(decoder_inputs)

#Decoder RNN
decoder_rnn = tf.keras.layers.LSTM(memory_size,
                                   dropout=0.2,
                                   recurrent_dropout=0.3,
                                   return_sequences=True,
                                   return_state=True)

#Decoder RNN Output, State initialization from Encoder states
#Output will be all hidden sequences, last 'h' state and last 'c' state
all_hidden_states_d,last_hidden_state_d,last_cell_state_decoder = decoder_rnn(decoder_embedding_output,
                                                                            initial_state=encoder_states)

#Output Layer
decoder_dense = tf.keras.layers.Dense(decoder_vocab_size + 1, #+1 to make sure one-hot encoding works for highest index value
                                      activation='softmax')

#Output of Dense layer
decoder_outputs = decoder_dense(all_hidden_states_d)

In [ ]:
#All hidden states
all_hidden_states_d

In [ ]:
#Predicted probabilities
decoder_outputs

### Build Model using both Encoder and Decoder

In [ ]:
#Build a Seq2Seq model -> Encoder + Decoder
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], #2 Inputs to the model
                              decoder_outputs) #Output of the model

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

### Train the model

In [ ]:
model.fit([encoder_input_data, decoder_input_data],
          decoder_target_one_hot,
          batch_size=64,
          epochs=10,
          validation_split=0.2)

### Save the model for later reuse

In [ ]:
model.save('seq2seq_training_translation.hd5')

# Building Model for Prediction

### Build the Encoder Model to predict Encoder States

In [ ]:
encoder_model = tf.keras.models.Model(encoder_inputs, #Padded input sequences
                                      encoder_states) #Hidden state and Cell state at last time step

In [ ]:
encoder_model.summary()

In [ ]:
encoder_model.output

### Build the Decoder Model
<p/>

<ol><li>Define Input for both 'h' state and 'c' state initialization </li>
<li>Get Decoder RNN outputs along with h and c state</li>
<li>Get Decoder Dense layer output</li>
        <li>Build Model</li></ol>

##### Step 1 - Define Input for both 'h' state and 'c' state initialization

In [ ]:
#Hidden state input
decoder_state_input_h = tf.keras.layers.Input(shape=(memory_size))

#Cell state input
decoder_state_input_c = tf.keras.layers.Input(shape=(memory_size))

#Putting it together
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

##### Step 2 - Get Decoder RNN outputs along with h and c state

In [ ]:
#Get Embedding layer output
x = decoder_embedding(decoder_inputs)

#We will use the layer which we trained earlier
rnn_outputs, state_h, state_c = decoder_rnn(x, initial_state=decoder_states_inputs)

#Why do we need this?
decoder_states = [state_h, state_c]

##### Step 3 - Get Decoder Dense layer output

In [ ]:
decoder_outputs = decoder_dense(rnn_outputs)

##### Step 4 - Build Decoder Model

In [ ]:
decoder_model = tf.keras.models.Model([decoder_inputs] + decoder_states_inputs,  #Model inputs
                                      [decoder_outputs] + decoder_states)

In [ ]:
decoder_model.summary()

# Predicting output from Seq2Seq model

##### Build a prediction function

In [ ]:
decoder_t.word_index['<start>']

In [ ]:
int_to_word_decoder[51]

In [ ]:
def decode_sentence(input_sequence):

    #Get the encoder state values - Sentence embedding
    decoder_initial_states_value = encoder_model.predict(input_seq)

    #Build a sequence with '<start>' - starting sequence for Decoder
    target_seq = np.zeros((1,1))
    target_seq[0][0] = decoder_t.word_index['<start>']

    #flag to check if prediction should be stopped
    stop_loop = False

    #Initialize predicted sentence
    predicted_sentence = ''

    num_of_predictions = 0

    #start the loop
    while not stop_loop:

        predicted_outputs, h, c = decoder_model.predict([target_seq] +
                                                        decoder_initial_states_value)

        #Get the predicted word index with highest probability
        predicted_output = np.argmax(predicted_outputs[0,-1,:])

        #Get the predicted word from predicter index
        predicted_word = int_to_word_decoder[predicted_output]

        #Check if prediction should stop
        if(predicted_word == '<end>' or num_of_predictions > max_decoder_seq_length):

            stop_loop = True
            continue

        num_of_predictions += 1

        #Updated predicted sentence
        if (len(predicted_sentence) == 0):
            predicted_sentence = predicted_word
        else:
            predicted_sentence = predicted_sentence + ' ' + predicted_word

        #Update target_seq to be the predicted word index
        target_seq[0][0] = predicted_output

        #Update initial states value for decoder
        decoder_initial_states_value = [h,c]


    return predicted_sentence

##### Call Prediction function on a random sentence

In [ ]:
#Generate a random number
start_num = np.random.randint(0, high=len(encoder_text) - 10)

#Predict model output for 5 sentences
for i in range(start_num, start_num + 5):
    input_seq = encoder_input_data[i : i+1]
    predicted_sentence = decode_sentence(input_seq)
    print('--------')
    print ('Input sentence: ', encoder_text[i])
    print ('Predicted sentence: ', predicted_sentence )

##### Save encoder and decoder model

In [ ]:
#Compile models to avoid error
encoder_model.compile(optimizer='adam',loss='categorical_crossentropy')
decoder_model.compile(optimizer='adam',loss='categorical_crossentropy')

#Save the models
encoder_model.save('seq2seq_encoder_eng_hin.hd5')  #Encoder model
decoder_model.save('seq2seq_decoder_eng_hin.hd5')  #Decoder model

##### Save encoder and decoder tokenizers

In [ ]:
import pickle

pickle.dump(encoder_t,open('encoder_tokenizer_eng','wb'))
pickle.dump(decoder_t,open('decoder_tokenizer_hin','wb'))

In [ ]:
!ls -l